In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math
import random
import glm

# Setting OpenGL

In [2]:
# Capturando eventos de teclado e mouse
cameraPos   = glm.vec3(0.0,  0.0,  4.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp
    
    cameraSpeed = 0.10
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

firstMouse = True
yaw = 0.0 
pitch = 0.0
lastX =  400
lastY =  400
        
def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.4
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

def set_window(width, height, name):
	# Inicializando janela
	glfw.init()
	glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
	window = glfw.create_window(width, height, name, None, None)
	glfw.make_context_current(window)
	    
	glfw.set_key_callback(window,key_event)
	glfw.set_cursor_pos_callback(window, mouse_event)
	glfw.set_mouse_button_callback(window,mouse_event) 

	return window

def set_GPU():
	# GLSL para Vertex Shader
	vertex_code = """
	        attribute vec3 position;
	        uniform mat4 model, view, projection;
            
	        void main(){
	            gl_Position = projection * view * model * vec4(position,1.0);
                // gl_Position = view * model * vec4(position,1.0);
	        }
	        """

	# GLSL para Fragment Shader
	fragment_code = """
	        uniform vec4 color;
	        void main(){
	            gl_FragColor = color;
	        }
	        """
	# Request a program and shader slots from GPU
	program  = glCreateProgram()
	vertex   = glCreateShader(GL_VERTEX_SHADER)
	fragment = glCreateShader(GL_FRAGMENT_SHADER)

	# Set shaders source
	glShaderSource(vertex, vertex_code)
	glShaderSource(fragment, fragment_code)

	# Compile shaders
	glCompileShader(vertex)
	if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
	    error = glGetShaderInfoLog(vertex).decode()
	    print(error)
	    raise RuntimeError("Erro de compilacao do Vertex Shader")

	# Compilando o Fragment Shader
	glCompileShader(fragment)
	if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
	    error = glGetShaderInfoLog(fragment).decode()
	    print(error)
	    raise RuntimeError("Erro de compilacao do Fragment Shader")

	# Attach shader objects to the program
	glAttachShader(program, vertex)
	glAttachShader(program, fragment)

	# Build program
	glLinkProgram(program)
	if not glGetProgramiv(program, GL_LINK_STATUS):
	    print(glGetProgramInfoLog(program))
	    raise RuntimeError('Linking error')
	    
	# Make program the default program
	glUseProgram(program)

	return program

def set_GPUBuffer( program, vertices ):
	# Request a buffer slot from GPU
	buffer = glGenBuffers(1)
	# Make this buffer the default one
	glBindBuffer(GL_ARRAY_BUFFER, buffer)

	# Upload data1
	glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
	glBindBuffer(GL_ARRAY_BUFFER, buffer)

	# Bind the position attribute
	# --------------------------------------
	stride = vertices.strides[0]
	offset = ctypes.c_void_p(0)

	loc = glGetAttribLocation(program, "position")
	glEnableVertexAttribArray(loc)

	glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

In [3]:
def model(angle, rot, scale, trans):
    
    rotVec, scaleVec, transVec = glm.vec3(rot),glm.vec3(scale),glm.vec3(trans)
    
    # Creates em 4x4 identity matrix
    mat_transform = glm.mat4(1.0)
    # Fiz upside down state
    mat_transform = glm.rotate(mat_transform, math.radians(90), glm.vec3(1, 0, 0))
    # Transformation
    if angle!=0 : mat_transform = glm.rotate(mat_transform, angle, rotVec)
    mat_transform = glm.scale(mat_transform, scaleVec)
    mat_transform = glm.translate(mat_transform, transVec)
    # Trans
    mat_transform = np.array(mat_transform).T
    
    return mat_transform

def view( cameraPos, cameraFront, cameraUp ):
    
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view).T
    
    return mat_view

def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}, vertices = [], texture_coords = [], faces = [], material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"):       ## para cada linha do arquivo .obj
        if line.startswith('#'): continue  ## ignora comentarios
        values = line.split()               # quebra a linha por espaço
        if not values: continue

        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])

        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces
    return model

In [4]:
# Sets the window
window = set_window(800, 800, "Exercicio7")

# Configure shaders and construct variables
program = set_GPU()

In [5]:
# Import the OBJ points
modelo = load_model_from_file( 'cube.obj' )
vertices_list = []    

### inserindo vertices do modelo no vetor de vertices
for face in modelo['faces']:
    for vertice_id in face[0]: vertices_list.append( modelo['vertices'][vertice_id-1] )

vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

In [6]:
# Sets the GPU Buffer
set_GPUBuffer( program, vertices )

loc_color = glGetUniformLocation(program, "color")

glfw.show_window(window)

d = 0.0

# Show
while not glfw.window_should_close(window):

    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # animate the teapot rotation
    d += .001 
    
    rot, scale, trans = (1, 1, 0), (.2, .2, .2), (0, 0, -1)
    global cameraPos, cameraFront, cameraUp
    
    # View 
    mat_view = view( cameraPos, cameraFront, cameraUp )    
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)
    
    # Transform
    mat_transform = model( d, rot, scale, trans ) 
    loc = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_transform)
    
    # Projection
    mat_projection = glm.perspective(glm.radians(45.0), 1, 1, 150.0)
    mat_projection = np.array(mat_projection)
    loc_proj = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_proj, 1, GL_TRUE, mat_projection)
    
    
    glPolygonMode(GL_BACK,GL_LINE)
    
    for triangle in range(0,len(vertices),3):
       
        random.seed( triangle )
        R = random.random()
        G = random.random()
        B = random.random()

        # The teapot is more likely to be blue and green
        glUniform4f(loc_color, R, G, B, 1.0) 
        
        glDrawArrays(GL_TRIANGLES, triangle, 3)     

    glfw.swap_buffers(window)

glfw.terminate()

TypeError: mouse_event() takes 3 positional arguments but 4 were given

In [ ]:
final_matrix = mat_projection * mat_view * mat_transform
print( final_matrix )

#print( np.matmul(final_matrix, vertices) )
#for i in range(3) : print(vertices[i])

for point in vertices:
    print(point)